In [41]:
import requests
import pandas as pd
import cv2
import urllib.request
import numpy as np
from matplotlib import pyplot as plt
#from keras.preprocessing import image

%matplotlib inline


%store -r WORKDIR

#if 'WORKDIR' not in dir():
WORKDIR = 'C:/Users/cammy/OneDrive/MIT IA/git/projeto_dogs/dogs_brand'



baseurl = 'https://dog.ceo/api/'

listurl = baseurl + 'breeds/list/all'
randomappend = '/images/random/'

data_raw_file = WORKDIR + '/Data/Raw/dogs.parquet'

maxBreeds = 4
maxImages = 50

img_h, img_w = 64, 64 # Altura e largura das imagens

#breeds = []

In [42]:
WORKDIR

'C:/Users/cammy/OneDrive/MIT IA/git/projeto_dogs/dogs_brand'

# Busca Lista de Raças

In [43]:
obj = requests.get(listurl).json()
allbreeds = pd.DataFrame.from_dict(obj)
allbreeds.head()

,message,status
affenpinscher,[],success
african,[],success
airedale,[],success
akita,[],success
appenzeller,[],success


In [44]:
breeds = allbreeds.index
if maxBreeds > 0:
    breeds = allbreeds [0:maxBreeds].index
    

# Ler imagens

In [45]:
def ler_imagem(url):

# For remote, use urllib, as per "http://stackoverflow.com/questions/21061814"
    req = urllib.request.urlopen(url)

    arr = np.asarray(bytearray(req.read()), dtype=np.uint8)

    img = cv2.imdecode(arr,0)

# The important part - Correct BGR to RGB channel
#    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    imres = cv2.resize(img, (img_h, img_w), interpolation=cv2.INTER_CUBIC)
#    imres = image.img_to_array(imres.T)
#    imres = np.expand_dims(imres, axis = 0)

# Plot
#    plt.figure()  
#    plt.imshow(imres)
    return imres

In [46]:
imageurls = []
dogimages = []
rotulos = []
#dataset = np.ndarray((len(breeds) * maxImages, img_h, img_w, 3), dtype=np.uint8)
i = 0

for breed in breeds:
    filesurl = baseurl + 'breed/' + breed + randomappend + str(maxImages)
    imageurl = requests.get(filesurl).json()
    print(imageurl)
    breedimages = pd.DataFrame.from_dict(imageurl['message'])
    imageurls.append(breedimages)
    for images in breedimages.values:
        img = ler_imagem(images[0])
#        dataset[i] = img
        rotulos.append(breed)
        dogimages.append(img.flatten())
        i = i+1

{'message': ['https://images.dog.ceo/breeds/affenpinscher/n02110627_10147.jpg', 'https://images.dog.ceo/breeds/affenpinscher/n02110627_10185.jpg', 'https://images.dog.ceo/breeds/affenpinscher/n02110627_10437.jpg', 'https://images.dog.ceo/breeds/affenpinscher/n02110627_10439.jpg', 'https://images.dog.ceo/breeds/affenpinscher/n02110627_10859.jpg', 'https://images.dog.ceo/breeds/affenpinscher/n02110627_11262.jpg', 'https://images.dog.ceo/breeds/affenpinscher/n02110627_11263.jpg', 'https://images.dog.ceo/breeds/affenpinscher/n02110627_11279.jpg', 'https://images.dog.ceo/breeds/affenpinscher/n02110627_11283.jpg', 'https://images.dog.ceo/breeds/affenpinscher/n02110627_11365.jpg', 'https://images.dog.ceo/breeds/affenpinscher/n02110627_11584.jpg', 'https://images.dog.ceo/breeds/affenpinscher/n02110627_11657.jpg', 'https://images.dog.ceo/breeds/affenpinscher/n02110627_11798.jpg', 'https://images.dog.ceo/breeds/affenpinscher/n02110627_11811.jpg', 'https://images.dog.ceo/breeds/affenpinscher/n021

In [47]:
rotulos

['affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'affenpinscher',
 'african',
 'african',
 'african',
 'african',
 'african',
 'african',
 'african',
 'african',
 'af

In [48]:
#dataset.shape

In [49]:
dogimages

[array([ 42,  41, 127, ..., 105, 128, 120], dtype=uint8),
 array([ 78,  77,  78, ..., 115, 113, 109], dtype=uint8),
 array([183, 184, 186, ..., 178, 161, 156], dtype=uint8),
 array([128, 143, 174, ...,  38,  42,  41], dtype=uint8),
 array([ 25,  13, 157, ...,  61,  51,  48], dtype=uint8),
 array([ 7,  8,  8, ..., 30, 11, 10], dtype=uint8),
 array([55, 57, 63, ..., 72, 50, 52], dtype=uint8),
 array([192, 187, 184, ..., 105, 143, 147], dtype=uint8),
 array([ 95, 118,  80, ..., 108, 123, 151], dtype=uint8),
 array([ 49,  25,  25, ..., 150, 158, 162], dtype=uint8),
 array([250, 232, 243, ..., 215, 203, 209], dtype=uint8),
 array([ 86, 158, 112, ...,  80,  86,  76], dtype=uint8),
 array([119,  14,  18, ..., 156, 165, 163], dtype=uint8),
 array([ 55, 138, 136, ..., 214, 214, 211], dtype=uint8),
 array([111, 135, 149, ..., 109, 103,  94], dtype=uint8),
 array([167, 166, 165, ..., 242, 242, 242], dtype=uint8),
 array([110, 128, 141, ..., 132, 138, 141], dtype=uint8),
 array([255, 255, 255, ...

In [50]:
# view an image (e.g. 20) and print its corresponding label
#img_index = 10
#plt.imshow(dataset[img_index,:,:,:])
#plt.show()
#print(rotulos[img_index])

In [51]:
df = pd.DataFrame(dogimages)
df['breed'] = rotulos

In [52]:
df

,0,1,2,3,4,5,6,7,8,9,...,4087,4088,4089,4090,4091,4092,4093,4094,4095,breed
0,42,41,127,125,49,111,98,16,54,178,...,99,93,97,92,102,97,105,128,120,affenpinscher
1,78,77,78,78,106,68,81,64,69,67,...,114,119,112,105,105,87,115,113,109,affenpinscher
2,183,184,186,186,189,191,200,78,86,157,...,178,180,160,162,140,154,178,161,156,affenpinscher
3,128,143,174,253,253,253,43,7,12,15,...,50,60,47,48,50,39,38,42,41,affenpinscher
4,25,13,157,73,70,65,60,39,54,38,...,26,84,74,64,65,55,61,51,48,affenpinscher
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,143,144,145,145,146,147,147,147,148,148,...,203,212,179,197,200,209,181,193,207,akita
155,77,72,70,84,61,44,35,26,46,56,...,192,202,210,226,103,100,93,208,151,akita
156,255,255,255,255,255,255,255,255,255,255,...,61,25,81,40,12,51,64,68,63,akita
157,146,148,150,152,154,156,156,157,158,165,...,188,187,179,175,174,171,168,164,156,akita


In [53]:
df.columns = df.columns.astype(str)
df.to_parquet(data_raw_file)